In [1]:
import importlib
from pathlib import Path
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

In [2]:
from bsm_basic_agent.config import llm_config
importlib.reload(llm_config)
from bsm_basic_agent.config.llm_config import get_llm

from bsm_basic_agent import tools
importlib.reload(tools)
from bsm_basic_agent.tools import get_tools_for_role

from bsm_basic_agent.agents import agent_factory
importlib.reload(agent_factory)
from bsm_basic_agent.agents.agent_factory import built_graph_agent

from bsm_basic_agent.prompts import loader
importlib.reload(loader)
from bsm_basic_agent.prompts.loader import load_prompt

In [3]:
agent_role = 'basic_bsm_agent'
tools = get_tools_for_role(agent_role)
tools

[StructuredTool(name='read_csv_records', description='Reads a CSV file from the specified filepath and returns the first five rows in JSON format without the index. If an error occurs during reading,\nreturns an error message.\n\nArgs:\n    filepath (str): The path to the CSV file to be read.\n\nReturns:\n    str: JSON string of the first five rows of the CSV file, or an error message.', args_schema=<class 'langchain_core.utils.pydantic.read_csv_records'>, func=<function csv_loader at 0x11262b7e0>),
 StructuredTool(name='calculate_bsm_price', description="Calculates the Black-Scholes price for European call or put options.\n\nArgs:\n    option_type (str): Type of the option, either 'call' or 'put'\n    S (float): Current price of the underlying asset\n    K (float): Strike price of the option\n    T (float): Time to expiration in years\n    r (float): Risk-free interest rate (annualized)\n    sigma (float): Volatility of the underlying asset (annualized standard deviation)\n\nReturns:\

In [4]:
llm = get_llm()
llm

ChatOllama(model='qwen2.5:7b', temperature=0.0)

In [5]:
agent1 = built_graph_agent(
    llm, tools
)

In [6]:
csv_path = Path.cwd().parents[1] / "data" / "input" / "dummy_options.csv"
prompt_path = Path.cwd().parents[1] / "src" / "bsm_basic_agent" / "prompts" / "basic_prompts.txt"
prompt = load_prompt(prompt_path).format(path=str(csv_path))
msg = HumanMessage(content=prompt)
print(f"Prompt:\n{msg.content}\n")
print(f"{'='*80}\n")

Prompt:
"Read ONLY the first 5 rows from file in \n\n"
"Path:/Users/yifanli/Library/CloudStorage/OneDrive-EY/github/model_doc_automation/data/input/dummy_options.csv\n\n"
"using csv_loader tool. After the data is successfully read from the file, "
"then calculate the Black-Scholes-Merton option price for ALL data points in ONE call. "
"\n\n"
"IMPORTANT: Use batch_bsm_calculator tool with the JSON data returned by csv_loader. "
"DO NOT call bsm_calculator multiple times for each row. "
"The batch_bsm_calculator will efficiently process all rows at once and return a formatted table."
"\n\n"
"Expected workflow:\n"
"1. Call csv_loader to read the CSV file\n"
"2. Pass the JSON result directly to batch_bsm_calculator\n"
"3. Display the markdown table returned by batch_bsm_calculator"




In [7]:
resp = agent1.invoke(
    {"messages": [msg]},
    config={
        "recursion_limit": 10,
        "configurable": {"thread_id": "run-1"}
    }
)

In [9]:
step_num = 1
for message in resp["messages"]:
    if isinstance(message, HumanMessage):
        print(f"Step {step_num} - inputs:")
        print(f"   {message.content[:200]}..." if len(message.content) > 200 else f"   {message.content}")
        print()
        step_num += 1

    elif isinstance(message, AIMessage):
        if hasattr(message, 'tool_calls') and message.tool_calls:
            # Agent 决定调用工具
            print(f"Step {step_num} - Agent decide tools used:")
            for tool_call in message.tool_calls:
                print(f"   Tool name: {tool_call['name']}")
                print(f"   Tool parameters: {tool_call['args']}")
            print()
            step_num += 1
        elif message.content:
            print(f"Step {step_num} - Agent outputs:")
            print(f"   {message.content}")
            print()
            step_num += 1

    elif isinstance(message, ToolMessage):
        print(f"Step {step_num} - outputs:")
        print(f"   Tool name: {message.name}")
        # result_preview = message.content[:300] + "..." if len(message.content) > 300 else message.content
        result_preview = message.content
        print(f"   Outputs: {result_preview}")
        print()
        step_num += 1

print(f"\n{'='*80}")
print(f"Final outputs:")
print(f"{'='*80}\n")
print(resp["messages"][-1].content)

Step 1 - inputs:
   "Read ONLY the first 5 rows from file in \n\n"
"Path:/Users/yifanli/Library/CloudStorage/OneDrive-EY/github/model_doc_automation/data/input/dummy_options.csv\n\n"
"using csv_loader tool. After the dat...

Step 2 - Agent decide tools used:
   Tool name: read_csv_records
   Tool parameters: {'filepath': '/Users/yifanli/Library/CloudStorage/OneDrive-EY/github/model_doc_automation/data/input/dummy_options.csv'}

Step 3 - outputs:
   Tool name: read_csv_records
   Outputs: {"date":{"0":"2025-09-01","1":"2025-09-02","2":"2025-09-03","3":"2025-09-04","4":"2025-09-05","5":"2025-09-06","6":"2025-09-07","7":"2025-09-08","8":"2025-09-09","9":"2025-09-10"},"S":{"0":100,"1":102,"2":98,"3":101,"4":99,"5":103,"6":97,"7":100,"8":104,"9":96},"K":{"0":105,"1":106,"2":104,"3":107,"4":103,"5":108,"6":102,"7":106,"8":109,"9":101},"T":{"0":1.0,"1":0.9,"2":0.8,"3":0.7,"4":0.6,"5":0.5,"6":0.4,"7":0.3,"8":0.2,"9":0.1},"r":{"0":0.05,"1":0.045,"2":0.048,"3":0.047,"4":0.046,"5":0.049,"6":0.044

In [1]:
import importlib
import sys
from pathlib import Path

from langchain_core.messages import HumanMessage, AIMessage, ToolMessage

In [3]:
from bsm_basic_agent.config import llm_config
importlib.reload(llm_config)
from bsm_basic_agent.config.llm_config import get_llm

In [4]:
from bsm_basic_agent.tools import basic_bsm_tools
importlib.reload(basic_bsm_tools)
from bsm_basic_agent.tools.basic_bsm_tools import get_tools

ImportError: cannot import name 'csv_loader' from 'core.bsm_calculator' (/Users/yifanli/Library/CloudStorage/OneDrive-EY/github/model_doc_automation/src/core/bsm_calculator.py)

In [ ]:


from src.agents.prompts import basic_prompts
importlib.reload(basic_prompts)
from src.agents.prompts.basic_prompts import READ_AND_CALC

from src.agents.nodes import basic_agent
importlib.reload(basic_agent)
from src.agents.nodes.basic_agent import build_agent

In [4]:
llm = get_llm()
tools = get_tools()
agent = build_agent(llm, tools)

In [5]:
csv_path = Path.cwd().parents[1] / "inputs" / "dummy_options.csv"
print(f"\n{'='*80}")
print(f"CSV path: {csv_path}")
print(f"{'='*80}\n")


CSV path: /Users/yifanli/Github/model_doc_automation/inputs/dummy_options.csv



In [6]:
msg = HumanMessage(content=READ_AND_CALC.format(path=str(csv_path)))
print(f"Prompt:\n{msg.content}\n")
print(f"{'='*80}\n")

Prompt:
Read ONLY the first 5 rows from file in 

Path:/Users/yifanli/Github/model_doc_automation/inputs/dummy_options.csv

using csv_loader tool. After the data is successfully read from the file, then calculate the Black-Scholes-Merton option price for ALL data points in ONE call. 

IMPORTANT: Use batch_bsm_calculator tool with the JSON data returned by csv_loader. DO NOT call bsm_calculator multiple times for each row. The batch_bsm_calculator will efficiently process all rows at once and return a formatted table.

Expected workflow:
1. Call csv_loader to read the CSV file
2. Pass the JSON result directly to batch_bsm_calculator
3. Display the markdown table returned by batch_bsm_calculator




In [7]:
resp = agent.invoke(
    {"messages": [msg]},
    config={
        "recursion_limit": 10,
        "configurable": {"thread_id": "run-1"}
    }
)

In [8]:
step_num = 1
for message in resp["messages"]:
    if isinstance(message, HumanMessage):
        print(f"Step {step_num} - inputs:")
        print(f"   {message.content[:200]}..." if len(message.content) > 200 else f"   {message.content}")
        print()
        step_num += 1

    elif isinstance(message, AIMessage):
        if hasattr(message, 'tool_calls') and message.tool_calls:
            # Agent 决定调用工具
            print(f"Step {step_num} - Agent decide tools used:")
            for tool_call in message.tool_calls:
                print(f"   Tool name: {tool_call['name']}")
                print(f"   Tool parameters: {tool_call['args']}")
            print()
            step_num += 1
        elif message.content:
            print(f"Step {step_num} - Agent outputs:")
            print(f"   {message.content}")
            print()
            step_num += 1

    elif isinstance(message, ToolMessage):
        print(f"Step {step_num} - outputs:")
        print(f"   Tool name: {message.name}")
        result_preview = message.content[:300] + "..." if len(message.content) > 300 else message.content
        print(f"   Outputs: {result_preview}")
        print()
        step_num += 1

print(f"\n{'='*80}")
print(f"Final outputs:")
print(f"{'='*80}\n")
print(resp["messages"][-1].content)

Step 1 - inputs:
   Read ONLY the first 5 rows from file in 

Path:/Users/yifanli/Github/model_doc_automation/inputs/dummy_options.csv

using csv_loader tool. After the data is successfully read from the file, then calcu...

Step 2 - Agent decide tools used:
   Tool name: csv_loader
   Tool parameters: {'filepath': '/Users/yifanli/Github/model_doc_automation/inputs/dummy_options.csv'}

Step 3 - outputs:
   Tool name: csv_loader
   Outputs: {"date":{"0":"2025-09-01","1":"2025-09-02","2":"2025-09-03","3":"2025-09-04","4":"2025-09-05"},"S":{"0":100,"1":102,"2":98,"3":101,"4":99},"K":{"0":105,"1":106,"2":104,"3":107,"4":103},"T":{"0":1.0,"1":0.9,"2":0.8,"3":0.7,"4":0.6},"r":{"0":0.05,"1":0.045,"2":0.048,"3":0.047,"4":0.046},"sigma":{"0":0...

Step 4 - Agent decide tools used:
   Tool name: batch_bsm_calculator
   Tool parameters: {'csv_data': {'date': ['2025-09-01', '2025-09-02', '2025-09-03', '2025-09-04', '2025-09-05'], 'S': [100, 102, 98, 101, 99], 'K': [105, 106, 104, 107, 103], 'T': [